In [ ]:
from transformers import AutoModel
import torch

model_path = "/root/.cache/huggingface/hub/models--THUDM--chatglm-6b/snapshots/658202d88ac4bb782b99e99ac3adff58b4d0b813"
model = AutoModel.from_pretrained(model_path, trust_remote_code=True, load_in_8bit=True, device_map='auto')

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [ ]:
from peft import PeftModel
peft_path = "/root/autodl-tmp/output"
model = PeftModel.from_pretrained(model, peft_path)

In [ ]:
import json

instructions = json.load(open("/root/FinGPT-ChatGLM-Fineturning/data/title/dataset_title_test.json"))

In [ ]:
import pandas as pd
instructions = pd.DataFrame(instructions)
instructions.head(2)

In [ ]:
from cover_alpaca2jsonl import format_example
instructions[["context","target"]] = instructions.apply(format_example, axis = 1, result_type="expand")

In [ ]:
instructions.head(2)

In [ ]:
# input_text_list = []
# out_list = []

# for i in range(total_steps):
#     tmp_context = context_list[i* batch_size:(i+1)* batch_size]
    
#     tmp_input_text = 
#     out = model.generate(
#         input_ids=tmp_input_text,
#         max_length=150,
#         do_sample=False,
#         temperature=0
#     )
#     out_text = tokenizer.decode(out[0])
#     input_text_list.append(tmp_input_text)
#     out_list.append(out_text)
#     break

In [ ]:
instructions

In [ ]:
instructions.context[0]

In [ ]:
def encode(x):
    ids = tokenizer.encode(x, max_length= 1024, return_tensors="np", padding = "max_length")
    input_ids = torch.LongTensor([ids])
    return input_ids
instructions["input_text"] = instructions.context.apply(encode)

In [ ]:
# instructions.to_pickle("instructions.pkl")

In [ ]:
# import pandas as pd
# instructions = pd.read_pickle("instructions.pkl")[:1000]

In [ ]:
instructions.shape

In [ ]:
batch_size = 16
total_steps = instructions.shape[0]//batch_size
total_steps

In [ ]:
input_text_list = instructions.input_text.tolist()
torch.cuda.empty_cache()

In [ ]:
from tqdm import tqdm
out_list = []
out_text_list = []

for i in tqdm(range(total_steps+1)):
    tmp_input_text = input_text_list[i* batch_size:(i+1)* batch_size]
    tmp_input_text = [i[0] for i in tmp_input_text]
    tmp_input_text = torch.concat(tmp_input_text, dim = 0)
    with torch.no_grad():
        out = model.generate(
            input_ids=tmp_input_text,
            max_length=128,
            do_sample=False,
            temperature=0
        )
    out = [tokenizer.decode(o) for o in out]
    out_text = [o.split("Answer: ")[1] for o in out]
    out_list += out
    out_text_list += out_text
    torch.cuda.empty_cache()

In [ ]:
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,recall_score,precision_score, auc
import pickle

In [ ]:
len(out_text_list)

In [ ]:
out_text_list_new =  [o.split("Answer: ")[1:] for o in out_list]

In [ ]:
out_list[4]

In [ ]:
out_text_list = [" ".join(o) for o in out_text_list]
# out_text_list

In [ ]:
a = ["replace", "Positive", "positive"]
a = [aa.replace("Positive", "positive") for aa in a]
a

In [ ]:
out_text_list = [o.replace("Positive", "positive") for o in out_text_list]
out_text_list

In [ ]:
import pandas as pd

In [ ]:
root_path = "/root/autodl-tmp/results"

In [ ]:
instructions = pd.read_csv(f"{root_path}/results_new.csv")
instructions.shape

In [ ]:
ori_instructions = pd.read_csv(f"{root_path}/results.csv")
ori_instructions.shape

In [ ]:
import json

instructions_ori =pd.read_csv("/root/FinGPT-ChatGLM-Fineturning/data/title/dataset_title_test.csv")

In [ ]:
instructions_ori.shape

In [ ]:
instructions.head(2)

In [ ]:
instructions_ori[["code_name"]]

In [ ]:
instructions.shape

In [ ]:
instructions = pd.concat([instructions, instructions_ori[["code_name"]]],axis = 1)
instructions.shape

In [ ]:
ori_instructions = pd.concat([ori_instructions, instructions_ori[["code_name"]]],axis = 1)
ori_instructions.shape

In [ ]:
instructions.to_csv(f"{root_path}/results_new.csv")
ori_instructions.to_csv(f"{root_path}/results.csv")

In [ ]:
accuracy_score(instructions["target"], instructions['new_pred'])

In [ ]:
accuracy_score(ori_instructions["target"], ori_instructions['new_pred'])

In [ ]:
instructions_no_neutral = instructions[instructions.target != "neutral"]
instructions_no_neutral.shape

In [ ]:
instructions_no_neutral_ori = ori_instructions[ori_instructions.target != "neutral"]
instructions_no_neutral_ori.shape

In [ ]:
f1_score(instructions_no_neutral["target"], instructions_no_neutral["new_pred"], average = "macro")

In [ ]:
f1_score(instructions_no_neutral_ori["target"], instructions_no_neutral_ori["new_pred"], average = "macro")

In [ ]:
f1_score(instructions_no_neutral["target"], instructions_no_neutral["new_pred"], average = "macro")

In [ ]:
f1_score(instructions["target"], out_text_list, average = "macro")

In [ ]:
f1_score(ori_instructions["target"], ori_instructions["new_pred"], average = "macro")

In [ ]:
for id, tmp in instructions.groupby("code_name"):
    print(id, end = " ")
    print(tmp.shape, end = " ")
    print(accuracy_score(tmp["target"], tmp['new_pred']))

In [ ]:
for id, tmp in ori_instructions.groupby("code_name"):
    print(id, end = " ")
    print(tmp.shape, end = " ")
    print(accuracy_score(tmp["target"], tmp['new_pred']))

In [ ]:
out_text_path = f"{root_path}/out_text_list_new.pkl"
with open(out_text_path, "wb") as f:
    pickle.dump(out_text_list, f)

In [ ]:
out_path = f"{root_path}/out_list_new.pkl"
with open(out_path, "wb") as f:
    pickle.dump(out_list, f)

In [ ]:
instructions["new_pred_out"] = out_list
instructions["new_pred"] = out_text_list

In [ ]:
instructions.head(2)

In [ ]:
result_path = f"{root_path}/results_new.csv"
instructions.to_csv(result_path,index = False)

In [ ]:
ori_instructions = pd.read_csv(f"{root_path}/results.csv")
ori_instructions.shape

In [ ]:
accuracy_score(ori_instructions["target"], ori_instructions["new_pred"])

In [ ]:
import json

instructions_ori = json.load(open("/root/FinGPT-ChatGLM-Fineturning/data/title/dataset_title_test.json"))

In [ ]:
answers = []
from cover_alpaca2jsonl import format_example


with torch.no_grad():
    for idx, item in enumerate(instructions_ori[:3]):
        feature = format_example(item)
        input_text = feature['context']
        ids = tokenizer.encode(input_text)
        input_ids = torch.LongTensor([ids])
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            do_sample=False,
            temperature=0
        )
        out_text = tokenizer.decode(out[0])
        answer = out_text.replace(input_text, "").replace("\nEND", "").strip()
        item['infer_answer'] = answer
        print(out_text)
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')
        answers.append({'index': idx, **item})

In [ ]:
input_ids